# Classification-1

2値分類用のサンプルデータであるBreast Cancerを読み込みます。<br>特徴量が全て数値データで欠損値もないキレイなサンプルデータです。

In [ ]:
# Load and return the breast cancer wisconsin dataset (classification).
# The breast cancer dataset is a classic and very easy binary classification dataset.
import pandas as pd
from sklearn.datasets import load_breast_cancer

# Set dataframe
dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# check the shape
print('--------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('--------------------------------------------------')
print(y.value_counts())
print('--------------------------------------------------')
print('y=0 means Marignant(悪性),y=1 means Benign(良性):')
print('--------------------------------------------------')
X.join(y).head()

k近傍法とロジスティック回帰の呼び出し方を学びましょう。KNNはneighborsというクラスから呼び出しています。ロジスティック回帰は重みベクトルと特徴量の一次線形結合をベースとしたアルゴリズムなので、線形モデルから呼び出せます。

In [ ]:
# import libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

回帰でも学びましたが、機械学習のアルゴリズムは特徴量ベクトルを標準化するのが普通です。実装は標準化処理とモデル学習（重みの最適化）のプロセスをひとまとめに記述できるパイプラインが便利なため呼び出します。

In [ ]:
# import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

回帰モデルの評価にはR2スコア等がありました。分類モデルの評価の場合、最もわかりやすいものが正解率（Accuracy）です。正解率は予測対象データ（スコアリングデータ）の全件をNとした時、正例を正しく正例と分類できた数TP、負例を正しく負例と分類できた数TFとすると、 以下の数式で表せます。<br><center>Accuracy = (TP + TF) / N</center><br>つまり、正例・負例を問わず正しく分類されたサンプル数の割合です。sklearnでは正解データ（正解ベクトル）と予測データ（予測ベクトル）を与えると、Accuracyを計算してくれる関数が存在します。それを以下で呼び込みましょう。

In [ ]:
# 分類モデルの評価指標計算のための関数の読込
from sklearn.metrics import accuracy_score

以下コードの各行の役割を理解できていることを確認して下さい。

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Holdout
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=1)
# set pipelines for two different algorithms
pipelines ={
    'knn_5': Pipeline([('scl',StandardScaler()),
                       ('est',KNeighborsClassifier())]),
    'knn_50': Pipeline([('scl',StandardScaler()),
                        ('est',KNeighborsClassifier(n_neighbors=50))]),
    'logistic': Pipeline([('scl',StandardScaler()),
                          ('est',LogisticRegression(random_state=1))])    
}

knn_5はデフォルトのK近傍法のパフォーマンス確認、pipe_knn_50は近傍数を増やしたときの性能確認用です。Kを増やすことで予測性能（正解率）がどのように変化するか考えてみましょう。pipe_logisticはロジスティック回帰の性能確認用です。kNNよりは良いだろうというのが通常の期待です。それでは各パイプラインを学習(fit)させましょう。正常終了すれば終了です。

In [ ]:
# optimize the parameters of each algorithms
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train,y_train)
    print(pipe_name, ': Fitting Done')

最後に、３つのパイプラインの分類器としての性能（正解率）を比較します。<br>まずpipelineのpredictの出力ベクトルを見て下さい。<br>y_pred in {0,1}で予測ラベルが付与されていることがわかります。

In [ ]:
pipelines['logistic'].predict(X_train)[:10]

今回は使いませんが、実務では予測ラベルの他に各クラスに分類される予測確率をよく使います。<b>予測確率を出力させたい場合はpredictをpredic_probaに変更するだけです。</b>最初の列はy=0と分類される予測確率、2番目の列はy=1と分類される予測確率です。

In [ ]:
pipelines['logistic'].predict_proba(X_train)[:10]

accuracy_scoreは第一引数に正解ラベル、第二引数に予測ラベル（確率ではない）を指定します。<br>今回はpredictで予測ラベルを与えましょう。

In [ ]:
from sklearn.metrics import accuracy_score

scores = {}
for pipe_name, pipeline in pipelines.items():
    scores[(pipe_name,'train')] = accuracy_score(y_train, pipeline.predict(X_train))
    scores[(pipe_name,'test')] = accuracy_score(y_test, pipeline.predict(X_test))

pd.Series(scores).unstack()

k近傍法のkを5から50に増やしましたが、性能が落ちてしまいました。多くのデータを見過ぎて平均値予測に近づき過ぎてしまったようです。予測値が無難過ぎてTrainスコアとTestスコアが近いのも特徴です。ロジスティック回帰は、k近傍法より3pt（0.956 to 0.982）程度、改善することを確認できました。ただし、どのように改善するか（しないか）はデータ次第ですので、代表的なアルゴリズムをいつでも試せる準備をしておくことが、プロジェクト進行において大切です。